In [ ]:
# =============================================================================
# CÉLULA 15: FUNÇÕES AUXILIARES PARA ANÁLISE MULTIDIMENSIONAL DE DAFs
# =============================================================================

print("\n" + "="*80)
print("PARTE 12: FUNÇÕES AUXILIARES - ANÁLISE MULTIDIMENSIONAL")
print("="*80)

import plotly.graph_objects as go
from typing import Dict, Tuple

# ══════════════════════════════════════════════════════════════════════════
# 1. FUNÇÕES DE COLORIZAÇÃO E EMOJIS
# ══════════════════════════════════════════════════════════════════════════

def get_color_indicador(nivel: str, invertido: bool = False) -> str:
    """
    Retorna cor hex baseada no nível do indicador.
    
    Args:
        nivel: Nível do indicador ('EXCELENTE', 'ALTO/BOM', 'MEDIO', 'BAIXO/ALTO', 'CRITICO')
        invertido: Se True, inverte a lógica de cores (para pendência e exclusão)
    
    Returns:
        String com código hexadecimal da cor
    """
    cores_normais = {
        'EXCELENTE': '#27ae60',  # Verde escuro
        'ALTO': '#2ecc71',       # Verde claro
        'BOM': '#2ecc71',        # Verde claro
        'MEDIO': '#f39c12',      # Laranja
        'BAIXO': '#e67e22',      # Laranja escuro
        'CRITICO': '#e74c3c'     # Vermelho
    }
    
    cores_invertidas = {
        'EXCELENTE': '#27ae60',  # Verde escuro
        'BOM': '#2ecc71',        # Verde claro
        'MEDIO': '#f39c12',      # Laranja
        'ALTO': '#e67e22',       # Laranja escuro
        'CRITICO': '#e74c3c'     # Vermelho
    }
    
    nivel_upper = nivel.upper()
    cores = cores_invertidas if invertido else cores_normais
    
    return cores.get(nivel_upper, '#95a5a6')  # Cinza como default

def get_emoji_indicador(indicador: str) -> str:
    """
    Retorna emoji para cada tipo de indicador.
    
    Args:
        indicador: Nome do indicador
    
    Returns:
        String com emoji correspondente
    """
    emojis = {
        'autonomia': '🎯',
        'pendencia': '⏳',
        'exclusao': '🗑️',
        'autuacao': '⚖️',
        'geral': '📊'
    }
    
    return emojis.get(indicador.lower(), '📈')

def get_cor_por_score(score: float) -> str:
    """
    Retorna cor baseada no score numérico (0-100).
    
    Args:
        score: Score entre 0 e 100
    
    Returns:
        String com código hexadecimal da cor
    """
    if score >= 80:
        return '#27ae60'  # Verde escuro
    elif score >= 60:
        return '#2ecc71'  # Verde claro
    elif score >= 40:
        return '#f39c12'  # Laranja
    elif score >= 20:
        return '#e67e22'  # Laranja escuro
    else:
        return '#e74c3c'  # Vermelho

# ══════════════════════════════════════════════════════════════════════════
# 2. FUNÇÃO PARA CALCULAR SCORE GERAL
# ══════════════════════════════════════════════════════════════════════════

def calcular_score_geral(autonomia: float, pendencia: float, 
                        exclusao: float, autuacao: float) -> float:
    """
    Calcula score geral ponderado baseado nos 4 indicadores.
    
    Pesos:
    - 35% Autonomia (mais importante)
    - 25% Pendência
    - 25% Exclusão
    - 15% Autuação
    
    Args:
        autonomia: Score de autonomia (0-100)
        pendencia: Score de pendência (0-100, já invertido)
        exclusao: Score de exclusão (0-100, já invertido)
        autuacao: Score de autuação (0-100, balanceado)
    
    Returns:
        Score geral ponderado (0-100)
    """
    score = (
        autonomia * 0.35 +
        pendencia * 0.25 +
        exclusao * 0.25 +
        autuacao * 0.15
    )
    
    return round_builtin(score, 2)

# ══════════════════════════════════════════════════════════════════════════
# 3. FUNÇÃO PARA IDENTIFICAR PERFIL DA DAF
# ══════════════════════════════════════════════════════════════════════════

def identificar_perfil_daf(scores: Dict[str, float]) -> Tuple[str, str]:
    """
    Identifica perfil da DAF baseado nos 4 scores.
    
    Args:
        scores: Dicionário com os 4 scores {'autonomia': X, 'pendencia': Y, ...}
    
    Returns:
        Tupla (perfil, descrição)
    """
    autonomia = scores.get('autonomia', 0)
    pendencia = scores.get('pendencia', 0)
    exclusao = scores.get('exclusao', 0)
    autuacao = scores.get('autuacao', 0)
    
    # Contar quantos scores estão críticos (< 40)
    criticos = sum_builtin([1 for s in [autonomia, pendencia, exclusao, autuacao] if s < 40])
    
    # Contar quantos scores estão excelentes (>= 80)
    excelentes = sum_builtin([1 for s in [autonomia, pendencia, exclusao, autuacao] if s >= 80])
    
    # Perfil: Autônoma e Eficiente
    if autonomia >= 80 and pendencia >= 70 and exclusao >= 70:
        return ("Autônoma e Eficiente", 
                "DAF com alta autonomia, baixa pendência e exclusão controlada")
    
    # Perfil: Alta Pendência
    elif pendencia < 40:
        return ("Alta Pendência", 
                "DAF com acúmulo excessivo de inconsistências não resolvidas")
    
    # Perfil: Exclusão Excessiva
    elif exclusao < 40:
        return ("Exclusão Excessiva", 
                "DAF com padrão de exclusões acima do esperado")
    
    # Perfil: Alta Autuação
    elif autuacao >= 80:
        return ("Alta Autuação", 
                "DAF com taxa de autuação elevada - foco em fiscalização")
    
    # Perfil: Múltiplos Problemas
    elif criticos >= 2:
        return ("Múltiplos Problemas", 
                "DAF necessita atenção em múltiplos indicadores")
    
    # Perfil: Equilibrada
    elif excelentes >= 3:
        return ("Equilibrada", 
                "DAF com bom desempenho em todos os indicadores")
    
    # Perfil: Em Desenvolvimento
    elif autonomia >= 60 and criticos == 0:
        return ("Em Desenvolvimento", 
                "DAF com desempenho regular, sem problemas críticos")
    
    # Perfil: Necessita Atenção
    else:
        return ("Necessita Atenção", 
                "DAF com desempenho abaixo da média em alguns indicadores")

# ══════════════════════════════════════════════════════════════════════════
# 4. FUNÇÃO PARA CRIAR RADAR CHART
# ══════════════════════════════════════════════════════════════════════════

def criar_radar_chart(df_daf: pd.DataFrame, id_equipe: int = None, 
                     mostrar_media: bool = True) -> go.Figure:
    """
    Cria radar chart (gráfico de aranha) com os 4 scores de uma DAF.
    
    Args:
        df_daf: DataFrame com dados da(s) DAF(s)
        id_equipe: ID da equipe específica (None para todas)
        mostrar_media: Se True, adiciona linha com a média estadual
    
    Returns:
        Objeto Figure do Plotly
    """
    fig = go.Figure()
    
    # Se id_equipe específico, filtrar
    if id_equipe is not None:
        df_plot = df_daf[df_daf['id_equipe'] == id_equipe].copy()
    else:
        df_plot = df_daf.copy()
    
    # Categorias do radar
    categorias = ['Autonomia', 'Pendência', 'Exclusão', 'Autuação']
    
    # Para cada DAF
    for idx, row in df_plot.iterrows():
        valores = [
            float(row.get('score_autonomia', 0)),
            float(row.get('score_pendencia', 0)),
            float(row.get('score_exclusao', 0)),
            float(row.get('score_autuacao', 0))
        ]
        
        # Fechar o polígono
        valores_fechado = valores + [valores[0]]
        categorias_fechado = categorias + [categorias[0]]
        
        fig.add_trace(go.Scatterpolar(
            r=valores_fechado,
            theta=categorias_fechado,
            fill='toself',
            name=f"DAF {row['id_equipe']}",
            line=dict(width=2),
            opacity=0.7
        ))
    
    # Adicionar média estadual se solicitado
    if mostrar_media and len(df_daf) > 0:
        media_valores = [
            float(df_daf['score_autonomia'].mean()),
            float(df_daf['score_pendencia'].mean()),
            float(df_daf['score_exclusao'].mean()),
            float(df_daf['score_autuacao'].mean())
        ]
        media_valores_fechado = media_valores + [media_valores[0]]
        
        fig.add_trace(go.Scatterpolar(
            r=media_valores_fechado,
            theta=categorias_fechado,
            fill='toself',
            name='Média SC',
            line=dict(color='gray', width=2, dash='dash'),
            opacity=0.3
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100],
                tickvals=[0, 25, 50, 75, 100],
                ticktext=['0', '25', '50', '75', '100']
            )
        ),
        showlegend=True,
        title="Radar Chart - Indicadores de Performance",
        height=600
    )
    
    return fig

# ══════════════════════════════════════════════════════════════════════════
# 5. FUNÇÃO PARA CRIAR BARRAS HORIZONTAIS COMPARATIVAS
# ══════════════════════════════════════════════════════════════════════════

def criar_barras_comparativas(df_daf: pd.DataFrame, top_n: int = 10) -> go.Figure:
    """
    Cria gráfico de barras horizontais com os 4 scores lado a lado.
    
    Args:
        df_daf: DataFrame com dados das DAFs
        top_n: Número de DAFs a mostrar
    
    Returns:
        Objeto Figure do Plotly
    """
    # Ordenar por score geral e pegar top N
    df_top = df_daf.nlargest(top_n, 'score_geral_ponderado').copy()
    
    fig = go.Figure()
    
    # Adicionar traço para cada indicador
    fig.add_trace(go.Bar(
        name='🎯 Autonomia',
        y=df_top['id_equipe'].astype(str),
        x=df_top['score_autonomia'],
        orientation='h',
        marker=dict(color='#2ecc71')
    ))
    
    fig.add_trace(go.Bar(
        name='⏳ Pendência',
        y=df_top['id_equipe'].astype(str),
        x=df_top['score_pendencia'],
        orientation='h',
        marker=dict(color='#3498db')
    ))
    
    fig.add_trace(go.Bar(
        name='🗑️ Exclusão',
        y=df_top['id_equipe'].astype(str),
        x=df_top['score_exclusao'],
        orientation='h',
        marker=dict(color='#9b59b6')
    ))
    
    fig.add_trace(go.Bar(
        name='⚖️ Autuação',
        y=df_top['id_equipe'].astype(str),
        x=df_top['score_autuacao'],
        orientation='h',
        marker=dict(color='#e67e22')
    ))
    
    fig.update_layout(
        title=f'Top {top_n} DAFs - Comparação dos 4 Indicadores',
        xaxis_title='Score (0-100)',
        yaxis_title='ID da Equipe/DAF',
        barmode='group',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    return fig

print("\n✅ Funções auxiliares criadas:")
print("  • get_color_indicador() - Colorização por nível")
print("  • get_emoji_indicador() - Emojis por indicador")
print("  • get_cor_por_score() - Cor por score numérico")
print("  • calcular_score_geral() - Score geral ponderado")
print("  • identificar_perfil_daf() - Perfil da DAF")
print("  • criar_radar_chart() - Gráfico de aranha")
print("  • criar_barras_comparativas() - Barras comparativas")

In [ ]:
# =============================================================================
# CÉLULA 16: CARREGAR E PREPARAR DADOS MULTIDIMENSIONAIS (CORRIGIDA)
# =============================================================================

print("\n" + "="*80)
print("PARTE 13: ANÁLISE MULTIDIMENSIONAL DE DAFs")
print("="*80)

# Carregar dados das DAFs com nova estrutura
print("\n📥 Carregando dados multidimensionais das DAFs...")

df_dafs_multi = spark.sql("""
SELECT 
    id_equipe,
    qtd_empresas_acompanhadas,
    qtd_contadores_acompanhados,
    qtd_total_inconsistencias,
    ROUND(valor_total_inconsistencias / 1000000, 2) AS valor_incons_milhoes,
    
    -- Taxas brutas
    ROUND(taxa_autonomia_pct, 2) AS taxa_autonomia_pct,
    ROUND(taxa_exclusao_pct, 2) AS taxa_exclusao_pct,
    ROUND(taxa_autuacao_pct, 2) AS taxa_autuacao_pct,
    ROUND(taxa_pendencia_pct, 2) AS taxa_pendencia_pct,
    
    -- INDICADOR DE AUTONOMIA
    ROUND(ind_autonomia_pct, 2) AS ind_autonomia_pct,
    ind_autonomia_nivel,
    ROUND(score_autonomia, 2) AS score_autonomia,
    
    -- INDICADOR DE PENDÊNCIA
    ROUND(ind_pendencia_pct, 2) AS ind_pendencia_pct,
    ind_pendencia_nivel,
    ROUND(score_pendencia, 2) AS score_pendencia,
    
    -- INDICADOR DE EXCLUSÃO
    ROUND(ind_exclusao_pct, 2) AS ind_exclusao_pct,
    ind_exclusao_nivel,
    ROUND(score_exclusao, 2) AS score_exclusao,
    
    -- INDICADOR DE AUTUAÇÃO
    ROUND(ind_autuacao_pct, 2) AS ind_autuacao_pct,
    ind_autuacao_nivel,
    ROUND(score_autuacao, 2) AS score_autuacao,
    
    -- CLASSIFICAÇÃO GERAL
    ROUND(score_geral_ponderado, 2) AS score_geral_ponderado,
    classificacao_geral,
    
    -- FLAGS DE ALERTA
    flag_alerta_autonomia_critica,
    flag_alerta_pendencia_alta,
    flag_alerta_exclusao_alta,
    flag_alerta_autuacao_alta,
    
    -- Metadados
    dt_carga
    
FROM niat.mlh_performance_dafs
ORDER BY score_geral_ponderado DESC
""").toPandas()

# Converter Decimals para float
colunas_numericas = [
    'score_autonomia', 'score_pendencia', 'score_exclusao', 'score_autuacao',
    'score_geral_ponderado', 'ind_autonomia_pct', 'ind_pendencia_pct',
    'ind_exclusao_pct', 'ind_autuacao_pct', 'taxa_autonomia_pct',
    'taxa_exclusao_pct', 'taxa_autuacao_pct', 'taxa_pendencia_pct',
    'valor_incons_milhoes'
]

for col in colunas_numericas:
    if col in df_dafs_multi.columns:
        df_dafs_multi[col] = pd.to_numeric(df_dafs_multi[col], errors='coerce')

print(f"✅ Dados carregados: {len(df_dafs_multi)} DAFs")

# Estatísticas descritivas dos 4 scores
print("\n📊 ESTATÍSTICAS DOS 4 INDICADORES:")
print("="*80)

stats_indicadores = df_dafs_multi[[
    'score_autonomia', 'score_pendencia', 'score_exclusao', 'score_autuacao', 'score_geral_ponderado'
]].describe()

print(stats_indicadores)

# Distribuição por níveis
print("\n📊 DISTRIBUIÇÃO POR NÍVEIS:")
print("="*80)

for indicador in ['autonomia', 'pendencia', 'exclusao', 'autuacao']:
    col_nivel = f'ind_{indicador}_nivel'
    print(f"\n{get_emoji_indicador(indicador)} {indicador.upper()}:")
    if col_nivel in df_dafs_multi.columns:
        dist = df_dafs_multi[col_nivel].value_counts().sort_index()
        for nivel, qtd in dist.items():
            pct = qtd / len(df_dafs_multi) * 100
            print(f"  {nivel:15} {qtd:3} DAFs ({pct:5.1f}%)")

# Distribuição classificação geral
print(f"\n📊 CLASSIFICAÇÃO GERAL:")
dist_geral = df_dafs_multi['classificacao_geral'].value_counts().sort_index()
for nivel, qtd in dist_geral.items():
    pct = qtd / len(df_dafs_multi) * 100
    print(f"  {nivel:15} {qtd:3} DAFs ({pct:5.1f}%)")

# Contar alertas
print("\n🚨 ALERTAS ATIVOS:")
print("="*80)
alertas = {
    'Autonomia Crítica': int(df_dafs_multi['flag_alerta_autonomia_critica'].sum()),
    'Pendência Alta': int(df_dafs_multi['flag_alerta_pendencia_alta'].sum()),
    'Exclusão Alta': int(df_dafs_multi['flag_alerta_exclusao_alta'].sum()),
    'Autuação Alta': int(df_dafs_multi['flag_alerta_autuacao_alta'].sum())
}

for nome, qtd in alertas.items():
    print(f"  • {nome}: {qtd} DAFs")

# Identificar perfis
print("\n🎭 IDENTIFICANDO PERFIS DAS DAFs:")
print("="*80)

perfis_lista = []
for idx, row in df_dafs_multi.iterrows():
    scores_dict = {
        'autonomia': float(row['score_autonomia']) if pd.notna(row['score_autonomia']) else 0,
        'pendencia': float(row['score_pendencia']) if pd.notna(row['score_pendencia']) else 0,
        'exclusao': float(row['score_exclusao']) if pd.notna(row['score_exclusao']) else 0,
        'autuacao': float(row['score_autuacao']) if pd.notna(row['score_autuacao']) else 0
    }
    perfil, descricao = identificar_perfil_daf(scores_dict)
    perfis_lista.append(perfil)

df_dafs_multi['perfil'] = perfis_lista

# Distribuição de perfis
dist_perfis = df_dafs_multi['perfil'].value_counts()
print("\nDistribuição de Perfis:")
for perfil, qtd in dist_perfis.items():
    pct = qtd / len(df_dafs_multi) * 100
    print(f"  • {perfil:25} {qtd:3} DAFs ({pct:5.1f}%)")

print("\n✅ Análise multidimensional preparada!")

In [ ]:
# =============================================================================
# CÉLULA 17: VISUALIZAÇÕES MULTIDIMENSIONAIS - PARTE 1
# =============================================================================

print("\n" + "="*80)
print("PARTE 14: VISUALIZAÇÕES MULTIDIMENSIONAIS - DISTRIBUIÇÕES")
print("="*80)

# 1. HISTOGRAMAS DOS 4 SCORES
print("\n📊 1. HISTOGRAMAS DE DISTRIBUIÇÃO DOS 4 SCORES")
print("-"*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribuição dos 4 Indicadores de Performance', fontsize=16, fontweight='bold')

indicadores = [
    ('score_autonomia', '🎯 Autonomia', '#2ecc71'),
    ('score_pendencia', '⏳ Pendência', '#3498db'),
    ('score_exclusao', '🗑️ Exclusão', '#9b59b6'),
    ('score_autuacao', '⚖️ Autuação', '#e67e22')
]

for idx, (col, titulo, cor) in enumerate(indicadores):
    ax = axes[idx // 2, idx % 2]
    
    dados = df_dafs_multi[col].dropna()
    
    ax.hist(dados, bins=20, color=cor, alpha=0.7, edgecolor='black')
    ax.axvline(dados.mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {dados.mean():.1f}')
    ax.axvline(dados.median(), color='orange', linestyle='--', linewidth=2, label=f'Mediana: {dados.median():.1f}')
    
    ax.set_xlabel('Score (0-100)')
    ax.set_ylabel('Frequência (nº de DAFs)')
    ax.set_title(titulo)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# 2. BOX PLOTS POR CLASSIFICAÇÃO GERAL
print("\n📊 2. BOX PLOTS DOS 4 SCORES POR CLASSIFICAÇÃO GERAL")
print("-"*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribuição dos Scores por Classificação Geral', fontsize=16, fontweight='bold')

for idx, (col, titulo, cor) in enumerate(indicadores):
    ax = axes[idx // 2, idx % 2]
    
    # Preparar dados
    data_plot = []
    labels_plot = []
    
    for classificacao in ['EXCELENTE', 'BOM', 'REGULAR', 'ATENCAO', 'CRITICO']:
        dados_class = df_dafs_multi[df_dafs_multi['classificacao_geral'] == classificacao][col].dropna()
        if len(dados_class) > 0:
            data_plot.append(dados_class)
            labels_plot.append(classificacao)
    
    if len(data_plot) > 0:
        bp = ax.boxplot(data_plot, labels=labels_plot, patch_artist=True)
        
        # Colorir boxes
        for patch in bp['boxes']:
            patch.set_facecolor(cor)
            patch.set_alpha(0.7)
        
        ax.set_ylabel('Score (0-100)')
        ax.set_xlabel('Classificação Geral')
        ax.set_title(titulo)
        ax.grid(True, alpha=0.3, axis='y')
        ax.tick_params(axis='x', rotation=45)
    else:
        ax.text(0.5, 0.5, 'Sem dados suficientes', ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

# 3. HEATMAP DE DISTRIBUIÇÃO DOS NÍVEIS
print("\n📊 3. HEATMAP - DISTRIBUIÇÃO DOS NÍVEIS POR INDICADOR")
print("-"*80)

# Criar matriz de contagem
niveis_ordem = ['EXCELENTE', 'ALTO', 'BOM', 'MEDIO', 'BAIXO', 'CRITICO']
indicadores_cols = ['ind_autonomia_nivel', 'ind_pendencia_nivel', 'ind_exclusao_nivel', 'ind_autuacao_nivel']
indicadores_nomes = ['Autonomia', 'Pendência', 'Exclusão', 'Autuação']

matriz_niveis = pd.DataFrame(index=niveis_ordem, columns=indicadores_nomes)

for col_idx, col in enumerate(indicadores_cols):
    for nivel in niveis_ordem:
        count = (df_dafs_multi[col] == nivel).sum()
        matriz_niveis.loc[nivel, indicadores_nomes[col_idx]] = count

# Converter para numérico e preencher NaN
matriz_niveis = matriz_niveis.apply(pd.to_numeric, errors='coerce').fillna(0)

plt.figure(figsize=(12, 8))
sns.heatmap(matriz_niveis, annot=True, fmt='.0f', cmap='RdYlGn_r', 
           linewidths=0.5, cbar_kws={'label': 'Quantidade de DAFs'})
plt.title('Distribuição dos Níveis por Indicador', fontsize=16, fontweight='bold')
plt.xlabel('Indicador')
plt.ylabel('Nível')
plt.tight_layout()
plt.show()

print("\n✅ Visualizações de distribuição concluídas!")

In [ ]:
# =============================================================================
# CÉLULA 18: VISUALIZAÇÕES MULTIDIMENSIONAIS - PARTE 2
# =============================================================================

print("\n" + "="*80)
print("PARTE 15: VISUALIZAÇÕES MULTIDIMENSIONAIS - COMPARAÇÕES")
print("="*80)

# 4. SCATTER PLOT MULTIDIMENSIONAL
print("\n📊 4. SCATTER PLOT: AUTONOMIA vs EXCLUSÃO")
print("-"*80)

fig = px.scatter(df_dafs_multi,
                x='score_autonomia',
                y='score_exclusao',
                size='qtd_total_inconsistencias',
                color='score_geral_ponderado',
                hover_name='id_equipe',
                hover_data=['perfil', 'classificacao_geral', 'score_pendencia', 'score_autuacao'],
                title='Relação entre Autonomia e Exclusão (tamanho = volume, cor = score geral)',
                labels={'score_autonomia': 'Score de Autonomia',
                       'score_exclusao': 'Score de Exclusão',
                       'score_geral_ponderado': 'Score Geral'},
                color_continuous_scale='RdYlGn',
                height=700)

fig.add_hline(y=50, line_dash="dash", line_color="gray", annotation_text="Linha de corte")
fig.add_vline(x=50, line_dash="dash", line_color="gray")

fig.show()

# 5. BARRAS COMPARATIVAS TOP 15
print("\n📊 5. BARRAS COMPARATIVAS - TOP 15 DAFs")
print("-"*80)

fig_barras = criar_barras_comparativas(df_dafs_multi, top_n=15)
fig_barras.show()

# 6. MATRIZ DE CORRELAÇÃO ENTRE OS 4 SCORES
print("\n📊 6. MATRIZ DE CORRELAÇÃO ENTRE OS 4 INDICADORES")
print("-"*80)

scores_corr = df_dafs_multi[['score_autonomia', 'score_pendencia', 'score_exclusao', 'score_autuacao']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(scores_corr, annot=True, fmt='.3f', cmap='coolwarm', 
           center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
           vmin=-1, vmax=1)
plt.title('Matriz de Correlação entre os 4 Indicadores', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCORRELAÇÕES MAIS FORTES:")
correlacoes_fortes = []
for i in range(len(scores_corr.columns)):
    for j in range(i+1, len(scores_corr.columns)):
        corr_value = scores_corr.iloc[i, j]
        if abs_builtin(corr_value) > 0.3:
            correlacoes_fortes.append({
                'indicador1': scores_corr.columns[i],
                'indicador2': scores_corr.columns[j],
                'correlacao': corr_value
            })

for item in sorted(correlacoes_fortes, key=lambda x: abs_builtin(x['correlacao']), reverse=True):
    print(f"  • {item['indicador1']} ↔ {item['indicador2']}: {item['correlacao']:.3f}")

print("\n✅ Visualizações comparativas concluídas!")

In [ ]:
# =============================================================================
# CÉLULA 19: RADAR CHARTS E ANÁLISE DE PERFIS
# =============================================================================

print("\n" + "="*80)
print("PARTE 16: RADAR CHARTS E ANÁLISE DE PERFIS")
print("="*80)

# 7. RADAR CHART - TOP 5 DAFs
print("\n📊 7. RADAR CHART - TOP 5 DAFs POR SCORE GERAL")
print("-"*80)

top_5_dafs = df_dafs_multi.nlargest(5, 'score_geral_ponderado')
fig_radar = criar_radar_chart(top_5_dafs, mostrar_media=True)
fig_radar.show()

# 8. RADAR CHART POR PERFIL (Médias)
print("\n📊 8. RADAR CHART - MÉDIAS POR PERFIL")
print("-"*80)

# Calcular médias por perfil
perfis_medias = df_dafs_multi.groupby('perfil')[[
    'score_autonomia', 'score_pendencia', 'score_exclusao', 'score_autuacao'
]].mean().reset_index()

# Criar radar para cada perfil
fig = go.Figure()

categorias = ['Autonomia', 'Pendência', 'Exclusão', 'Autuação']

for idx, row in perfis_medias.iterrows():
    valores = [
        float(row['score_autonomia']),
        float(row['score_pendencia']),
        float(row['score_exclusao']),
        float(row['score_autuacao'])
    ]
    
    valores_fechado = valores + [valores[0]]
    categorias_fechado = categorias + [categorias[0]]
    
    fig.add_trace(go.Scatterpolar(
        r=valores_fechado,
        theta=categorias_fechado,
        fill='toself',
        name=row['perfil'],
        line=dict(width=2),
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100],
            tickvals=[0, 25, 50, 75, 100]
        )
    ),
    showlegend=True,
    title="Radar Chart - Médias por Perfil de DAF",
    height=700
)

fig.show()

# 9. ANÁLISE DETALHADA POR PERFIL
print("\n📊 9. ANÁLISE DETALHADA POR PERFIL")
print("="*80)

for perfil in df_dafs_multi['perfil'].unique():
    dafs_perfil = df_dafs_multi[df_dafs_multi['perfil'] == perfil]
    
    print(f"\n{'─'*80}")
    print(f"PERFIL: {perfil} ({len(dafs_perfil)} DAFs)")
    print(f"{'─'*80}")
    
    # Estatísticas do perfil
    print(f"\n📊 Scores Médios:")
    print(f"  🎯 Autonomia: {dafs_perfil['score_autonomia'].mean():.1f}")
    print(f"  ⏳ Pendência: {dafs_perfil['score_pendencia'].mean():.1f}")
    print(f"  🗑️ Exclusão: {dafs_perfil['score_exclusao'].mean():.1f}")
    print(f"  ⚖️ Autuação: {dafs_perfil['score_autuacao'].mean():.1f}")
    print(f"  📊 Geral: {dafs_perfil['score_geral_ponderado'].mean():.1f}")
    
    # Alertas ativos
    alertas_perfil = {
        'Autonomia Crítica': int(dafs_perfil['flag_alerta_autonomia_critica'].sum()),
        'Pendência Alta': int(dafs_perfil['flag_alerta_pendencia_alta'].sum()),
        'Exclusão Alta': int(dafs_perfil['flag_alerta_exclusao_alta'].sum()),
        'Autuação Alta': int(dafs_perfil['flag_alerta_autuacao_alta'].sum())
    }
    
    print(f"\n🚨 Alertas:")
    for nome_alerta, qtd_alerta in alertas_perfil.items():
        if qtd_alerta > 0:
            print(f"  • {nome_alerta}: {qtd_alerta} DAFs")
    
    # Top 3 DAFs do perfil
    if len(dafs_perfil) >= 3:
        print(f"\n🏆 Top 3 DAFs deste perfil:")
        top_3 = dafs_perfil.nlargest(3, 'score_geral_ponderado')
        for _, daf in top_3.iterrows():
            print(f"  • DAF {daf['id_equipe']}: Score {daf['score_geral_ponderado']:.1f} - {daf['classificacao_geral']}")

print("\n✅ Análise de perfis concluída!")

In [ ]:
# =============================================================================
# CÉLULA 20: CLUSTERING E SEGMENTAÇÃO MULTIDIMENSIONAL
# =============================================================================

print("\n" + "="*80)
print("PARTE 17: CLUSTERING E SEGMENTAÇÃO BASEADA NOS 4 INDICADORES")
print("="*80)

# Preparar dados para clustering
print("\n🔬 Preparando dados para clustering...")

features_clustering = ['score_autonomia', 'score_pendencia', 'score_exclusao', 'score_autuacao']
X_clustering = df_dafs_multi[features_clustering].fillna(df_dafs_multi[features_clustering].mean())

# Normalizar
scaler_dafs = StandardScaler()
X_clustering_scaled = scaler_dafs.fit_transform(X_clustering)

print(f"✅ Dados preparados: {len(X_clustering)} DAFs, {len(features_clustering)} features")

# Determinar número ótimo de clusters
print("\n📊 Determinando número ótimo de clusters...")

inertias_dafs = []
silhouette_scores_dafs = []
K_range_dafs = range(2, 11)

for k in K_range_dafs:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_clustering_scaled)
    inertias_dafs.append(kmeans.inertia_)
    silhouette_scores_dafs.append(silhouette_score(X_clustering_scaled, kmeans.labels_))

# Visualizar método Elbow
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Determinação do Número Ótimo de Clusters - DAFs', fontsize=16, fontweight='bold')

ax1.plot(K_range_dafs, inertias_dafs, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Número de Clusters (k)')
ax1.set_ylabel('Inércia')
ax1.set_title('Método Elbow')
ax1.grid(True, alpha=0.3)

ax2.plot(K_range_dafs, silhouette_scores_dafs, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Número de Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score por k')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Escolher k ótimo
k_otimo_dafs = K_range_dafs[np.argmax(silhouette_scores_dafs)]
max_silhouette_dafs = silhouette_scores_dafs[np.argmax(silhouette_scores_dafs)]

print(f"\n✅ Número ótimo de clusters: {k_otimo_dafs} (Silhouette: {max_silhouette_dafs:.3f})")

# Aplicar K-Means com k ótimo
print(f"\n🤖 Aplicando K-Means com k={k_otimo_dafs}...")

kmeans_final_dafs = KMeans(n_clusters=k_otimo_dafs, random_state=42, n_init=10)
df_dafs_multi['cluster_ml'] = kmeans_final_dafs.fit_predict(X_clustering_scaled)

# Analisar características dos clusters
print(f"\n📊 CARACTERÍSTICAS DOS {k_otimo_dafs} CLUSTERS:")
print("="*80)

for cluster_id in range(k_otimo_dafs):
    cluster_data = df_dafs_multi[df_dafs_multi['cluster_ml'] == cluster_id]
    
    print(f"\n🔹 CLUSTER {cluster_id} ({len(cluster_data)} DAFs - {len(cluster_data)/len(df_dafs_multi)*100:.1f}%)")
    print("-"*80)
    
    # Scores médios
    print(f"  📊 Scores Médios:")
    print(f"    🎯 Autonomia: {cluster_data['score_autonomia'].mean():.1f}")
    print(f"    ⏳ Pendência: {cluster_data['score_pendencia'].mean():.1f}")
    print(f"    🗑️ Exclusão: {cluster_data['score_exclusao'].mean():.1f}")
    print(f"    ⚖️ Autuação: {cluster_data['score_autuacao'].mean():.1f}")
    print(f"    📈 Geral: {cluster_data['score_geral_ponderado'].mean():.1f}")
    
    # Classificação geral predominante
    class_predominante = cluster_data['classificacao_geral'].mode()
    if len(class_predominante) > 0:
        print(f"  🏷️ Classificação predominante: {class_predominante.iloc[0]}")
    
    # Perfil predominante
    perfil_predominante = cluster_data['perfil'].mode()
    if len(perfil_predominante) > 0:
        print(f"  🎭 Perfil predominante: {perfil_predominante.iloc[0]}")
    
    # Alertas ativos
    total_alertas = (
        cluster_data['flag_alerta_autonomia_critica'].sum() +
        cluster_data['flag_alerta_pendencia_alta'].sum() +
        cluster_data['flag_alerta_exclusao_alta'].sum() +
        cluster_data['flag_alerta_autuacao_alta'].sum()
    )
    print(f"  🚨 Total de alertas: {int(total_alertas)}")
    
    # DAFs de exemplo
    print(f"  📋 Exemplos de DAFs:")
    exemplos = cluster_data.nlargest(3, 'score_geral_ponderado')['id_equipe'].tolist()
    print(f"    {exemplos}")

# Visualização com PCA
print("\n📊 Visualizando clusters com PCA...")

pca_dafs = PCA(n_components=2)
X_pca_dafs = pca_dafs.fit_transform(X_clustering_scaled)

plt.figure(figsize=(14, 8))
scatter = plt.scatter(X_pca_dafs[:, 0], X_pca_dafs[:, 1], 
                     c=df_dafs_multi['cluster_ml'],
                     cmap='viridis', s=100, alpha=0.6, 
                     edgecolors='w', linewidth=0.5)
plt.colorbar(scatter, label='Cluster')
plt.xlabel(f'PC1 ({pca_dafs.explained_variance_ratio_[0]:.1%} variância)')
plt.ylabel(f'PC2 ({pca_dafs.explained_variance_ratio_[1]:.1%} variância)')
plt.title('Clustering de DAFs baseado nos 4 Indicadores (K-Means + PCA)', 
         fontsize=16, fontweight='bold')

# Adicionar labels para algumas DAFs
for idx in df_dafs_multi.nlargest(5, 'score_geral_ponderado').index:
    plt.annotate(f"DAF {df_dafs_multi.loc[idx, 'id_equipe']}", 
                (X_pca_dafs[idx, 0], X_pca_dafs[idx, 1]),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Clustering concluído!")

In [ ]:
# =============================================================================
# CÉLULA 21: ANÁLISE DE ALERTAS E PRIORIZAÇÃO
# =============================================================================

print("\n" + "="*80)
print("PARTE 18: ANÁLISE DE ALERTAS E PRIORIZAÇÃO DE AÇÕES")
print("="*80)

# 1. DAFs com múltiplos alertas
print("\n🚨 1. DAFs COM MÚLTIPLOS ALERTAS")
print("-"*80)

df_dafs_multi['total_alertas'] = (
    df_dafs_multi['flag_alerta_autonomia_critica'] +
    df_dafs_multi['flag_alerta_pendencia_alta'] +
    df_dafs_multi['flag_alerta_exclusao_alta'] +
    df_dafs_multi['flag_alerta_autuacao_alta']
)

dafs_multiplos_alertas = df_dafs_multi[df_dafs_multi['total_alertas'] >= 2].sort_values(
    'total_alertas', ascending=False
)

if len(dafs_multiplos_alertas) > 0:
    print(f"\n⚠️ {len(dafs_multiplos_alertas)} DAFs com 2 ou mais alertas:")
    print("\nTop 10:")
    
    for idx, row in dafs_multiplos_alertas.head(10).iterrows():
        alertas_ativos = []
        if row['flag_alerta_autonomia_critica']:
            alertas_ativos.append('Autonomia Crítica')
        if row['flag_alerta_pendencia_alta']:
            alertas_ativos.append('Pendência Alta')
        if row['flag_alerta_exclusao_alta']:
            alertas_ativos.append('Exclusão Alta')
        if row['flag_alerta_autuacao_alta']:
            alertas_ativos.append('Autuação Alta')
        
        print(f"\n  🔴 DAF {row['id_equipe']}:")
        print(f"     Score Geral: {row['score_geral_ponderado']:.1f}")
        print(f"     Classificação: {row['classificacao_geral']}")
        print(f"     Alertas: {', '.join(alertas_ativos)}")
else:
    print("✅ Nenhuma DAF com múltiplos alertas!")

# 2. Matriz de Priorização
print("\n\n📊 2. MATRIZ DE PRIORIZAÇÃO (SCORE vs ALERTAS)")
print("-"*80)

fig = px.scatter(df_dafs_multi,
                x='score_geral_ponderado',
                y='total_alertas',
                size='qtd_total_inconsistencias',
                color='classificacao_geral',
                hover_name='id_equipe',
                hover_data=['perfil', 'score_autonomia', 'score_pendencia'],
                title='Matriz de Priorização: Score Geral vs Número de Alertas',
                labels={'score_geral_ponderado': 'Score Geral Ponderado',
                       'total_alertas': 'Número de Alertas Ativos'},
                color_discrete_map={
                    'EXCELENTE': '#27ae60',
                    'BOM': '#2ecc71',
                    'REGULAR': '#f39c12',
                    'ATENCAO': '#e67e22',
                    'CRITICO': '#e74c3c'
                },
                height=700)

# Adicionar linhas de divisão
fig.add_hline(y=1.5, line_dash="dash", line_color="red", annotation_text="Linha de Alerta")
fig.add_vline(x=50, line_dash="dash", line_color="red", annotation_text="Score Mínimo")

fig.show()

# 3. Recomendações de Ação por Categoria
print("\n\n💡 3. RECOMENDAÇÕES DE AÇÃO POR CATEGORIA")
print("="*80)

# Categoria 1: Críticas (múltiplos alertas + score baixo)
criticas = df_dafs_multi[
    (df_dafs_multi['total_alertas'] >= 2) & 
    (df_dafs_multi['score_geral_ponderado'] < 40)
]

print(f"\n🔴 CATEGORIA 1: CRÍTICAS ({len(criticas)} DAFs)")
print("   Ação: INTERVENÇÃO IMEDIATA")
if len(criticas) > 0:
    print(f"   DAFs: {criticas['id_equipe'].tolist()}")

# Categoria 2: Atenção (1 alerta ou score médio)
atencao = df_dafs_multi[
    ((df_dafs_multi['total_alertas'] == 1) | 
     ((df_dafs_multi['score_geral_ponderado'] >= 40) & 
      (df_dafs_multi['score_geral_ponderado'] < 60))) &
    (~df_dafs_multi['id_equipe'].isin(criticas['id_equipe']))
]

print(f"\n🟡 CATEGORIA 2: NECESSITAM ATENÇÃO ({len(atencao)} DAFs)")
print("   Ação: MONITORAMENTO REFORÇADO")
if len(atencao) > 5:
    print(f"   Exemplos: {atencao['id_equipe'].head(5).tolist()}")

# Categoria 3: Boas (score alto, poucos alertas)
boas = df_dafs_multi[
    (df_dafs_multi['total_alertas'] == 0) & 
    (df_dafs_multi['score_geral_ponderado'] >= 70)
]

print(f"\n🟢 CATEGORIA 3: BOAS PRÁTICAS ({len(boas)} DAFs)")
print("   Ação: BENCHMARKING E RECONHECIMENTO")
if len(boas) > 5:
    print(f"   Exemplos: {boas['id_equipe'].head(5).tolist()}")

# 4. Plano de Ação por Indicador
print("\n\n📋 4. PLANO DE AÇÃO POR INDICADOR")
print("="*80)

# Autonomia Crítica
autonomia_critica = df_dafs_multi[df_dafs_multi['flag_alerta_autonomia_critica'] == 1]
if len(autonomia_critica) > 0:
    print(f"\n🎯 AUTONOMIA CRÍTICA ({len(autonomia_critica)} DAFs):")
    print(f"   Ações recomendadas:")
    print(f"   • Treinamento intensivo para contadores")
    print(f"   • Revisão de processos de regularização")
    print(f"   • Suporte técnico adicional")
    print(f"   DAFs afetadas: {autonomia_critica['id_equipe'].tolist()[:10]}")

# Pendência Alta
pendencia_alta = df_dafs_multi[df_dafs_multi['flag_alerta_pendencia_alta'] == 1]
if len(pendencia_alta) > 0:
    print(f"\n⏳ PENDÊNCIA ALTA ({len(pendencia_alta)} DAFs):")
    print(f"   Ações recomendadas:")
    print(f"   • Força-tarefa para redução de backlog")
    print(f"   • Revisão de prazos e prioridades")
    print(f"   • Alocação de recursos adicionais")
    print(f"   DAFs afetadas: {pendencia_alta['id_equipe'].tolist()[:10]}")

# Exclusão Alta
exclusao_alta = df_dafs_multi[df_dafs_multi['flag_alerta_exclusao_alta'] == 1]
if len(exclusao_alta) > 0:
    print(f"\n🗑️ EXCLUSÃO ALTA ({len(exclusao_alta)} DAFs):")
    print(f"   Ações recomendadas:")
    print(f"   • Auditoria de padrões de exclusão")
    print(f"   • Treinamento em análise de legitimidade")
    print(f"   • Revisão de casos excluídos")
    print(f"   DAFs afetadas: {exclusao_alta['id_equipe'].tolist()[:10]}")

# Autuação Alta
autuacao_alta = df_dafs_multi[df_dafs_multi['flag_alerta_autuacao_alta'] == 1]
if len(autuacao_alta) > 0:
    print(f"\n⚖️ AUTUAÇÃO ALTA ({len(autuacao_alta)} DAFs):")
    print(f"   Ações recomendadas:")
    print(f"   • Análise de efetividade das autuações")
    print(f"   • Balanceamento com outras ações")
    print(f"   • Revisão de critérios de fiscalização")
    print(f"   DAFs afetadas: {autuacao_alta['id_equipe'].tolist()[:10]}")

print("\n✅ Análise de alertas e priorização concluída!")

In [ ]:
# =============================================================================
# CÉLULA 22: DASHBOARD EXECUTIVO MULTIDIMENSIONAL
# =============================================================================

print("\n" + "="*80)
print("PARTE 19: DASHBOARD EXECUTIVO MULTIDIMENSIONAL")
print("="*80)

# Criar visualização completa em painel único
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        '📊 Distribuição de Scores',
        '🎯 Classificação Geral',
        '🚨 Alertas Ativos por Indicador',
        '🎭 Distribuição de Perfis',
        '📈 Score Geral vs Autonomia',
        '🔢 Estatísticas Gerais'
    ),
    specs=[
        [{"type": "bar"}, {"type": "pie"}],
        [{"type": "bar"}, {"type": "pie"}],
        [{"type": "scatter"}, {"type": "table"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.15
)

# 1. Distribuição de Scores Médios
scores_medios = {
    'Autonomia': df_dafs_multi['score_autonomia'].mean(),
    'Pendência': df_dafs_multi['score_pendencia'].mean(),
    'Exclusão': df_dafs_multi['score_exclusao'].mean(),
    'Autuação': df_dafs_multi['score_autuacao'].mean()
}

fig.add_trace(
    go.Bar(x=list(scores_medios.keys()), y=list(scores_medios.values()),
          marker_color=['#2ecc71', '#3498db', '#9b59b6', '#e67e22'],
          text=[f'{v:.1f}' for v in scores_medios.values()],
          textposition='outside'),
    row=1, col=1
)

# 2. Classificação Geral
class_dist = df_dafs_multi['classificacao_geral'].value_counts()
fig.add_trace(
    go.Pie(labels=class_dist.index, values=class_dist.values,
          marker=dict(colors=['#27ae60', '#2ecc71', '#f39c12', '#e67e22', '#e74c3c'])),
    row=1, col=2
)

# 3. Alertas por Indicador
alertas_por_ind = {
    'Autonomia': int(df_dafs_multi['flag_alerta_autonomia_critica'].sum()),
    'Pendência': int(df_dafs_multi['flag_alerta_pendencia_alta'].sum()),
    'Exclusão': int(df_dafs_multi['flag_alerta_exclusao_alta'].sum()),
    'Autuação': int(df_dafs_multi['flag_alerta_autuacao_alta'].sum())
}

fig.add_trace(
    go.Bar(x=list(alertas_por_ind.keys()), y=list(alertas_por_ind.values()),
          marker_color='#e74c3c'),
    row=2, col=1
)

# 4. Distribuição de Perfis
perfil_dist = df_dafs_multi['perfil'].value_counts()
fig.add_trace(
    go.Pie(labels=perfil_dist.index, values=perfil_dist.values),
    row=2, col=2
)

# 5. Scatter: Score Geral vs Autonomia
fig.add_trace(
    go.Scatter(
        x=df_dafs_multi['score_autonomia'],
        y=df_dafs_multi['score_geral_ponderado'],
        mode='markers',
        marker=dict(
            size=8,
            color=df_dafs_multi['total_alertas'],
            colorscale='Reds',
            showscale=True,
            colorbar=dict(title="Alertas")
        ),
        text=df_dafs_multi['id_equipe'],
        hovertemplate='DAF %{text}<br>Autonomia: %{x:.1f}<br>Score Geral: %{y:.1f}<extra></extra>'
    ),
    row=3, col=1
)

# 6. Tabela de Estatísticas
stats_table = pd.DataFrame({
    'Métrica': [
        'Total de DAFs',
        'Score Geral Médio',
        'DAFs Excelentes',
        'DAFs Críticas',
        'Total de Alertas',
        'DAFs sem Alertas'
    ],
    'Valor': [
        len(df_dafs_multi),
        f"{df_dafs_multi['score_geral_ponderado'].mean():.1f}",
        len(df_dafs_multi[df_dafs_multi['classificacao_geral'] == 'EXCELENTE']),
        len(df_dafs_multi[df_dafs_multi['classificacao_geral'] == 'CRITICO']),
        int(df_dafs_multi['total_alertas'].sum()),
        len(df_dafs_multi[df_dafs_multi['total_alertas'] == 0])
    ]
})

fig.add_trace(
    go.Table(
        header=dict(values=list(stats_table.columns),
                   fill_color='#3498db',
                   align='left',
                   font=dict(color='white', size=12)),
        cells=dict(values=[stats_table['Métrica'], stats_table['Valor']],
                  fill_color='lavender',
                  align='left')
    ),
    row=3, col=2
)

# Atualizar layout
fig.update_layout(
    title_text="Dashboard Executivo - Análise Multidimensional de DAFs",
    title_font_size=20,
    showlegend=False,
    height=1400
)

fig.update_xaxes(title_text="Indicador", row=1, col=1)
fig.update_yaxes(title_text="Score Médio", row=1, col=1)

fig.update_xaxes(title_text="Indicador", row=2, col=1)
fig.update_yaxes(title_text="Nº de DAFs", row=2, col=1)

fig.update_xaxes(title_text="Score de Autonomia", row=3, col=1)
fig.update_yaxes(title_text="Score Geral", row=3, col=1)

fig.show()

print("\n✅ Dashboard executivo gerado!")

# Salvar resumo em variável para fácil acesso
resumo_multidimensional = {
    'total_dafs': len(df_dafs_multi),
    'score_geral_medio': df_dafs_multi['score_geral_ponderado'].mean(),
    'scores_medios': scores_medios,
    'distribuicao_classificacao': class_dist.to_dict(),
    'distribuicao_perfis': perfil_dist.to_dict(),
    'alertas_ativos': alertas_por_ind,
    'dafs_criticas': len(criticas) if 'criticas' in locals() else 0,
    'dafs_excelentes': len(boas) if 'boas' in locals() else 0
}

print("\n📊 RESUMO MULTIDIMENSIONAL SALVO EM: resumo_multidimensional")